In [1]:
import os

# Make sure the agents folder exists
os.makedirs("agents", exist_ok=True)
print("agents/ folder is ready!")

agents/ folder is ready!


In [2]:
%%writefile agents/prediction_agent.py
import joblib
import numpy as np

class PredictionAgent:
    def __init__(self, model_path):
        self.model = joblib.load(model_path)

    def predict_pm25(self, features):
        features = np.array(features).reshape(1, -1)
        prediction = self.model.predict(features)[0]
        return prediction

Overwriting agents/prediction_agent.py


In [3]:
# Create decision_agent.py
import os

# Ensure folder exists
os.makedirs("agents", exist_ok=True)

# Content of the file
code = """class DecisionAgent:
    def classify_aqi(self, pm25):
        if pm25 <= 12:
            return "Good 🟢"
        elif pm25 <= 35:
            return "Moderate 🟡"
        elif pm25 <= 55:
            return "Unhealthy 🟠"
        else:
            return "Hazardous 🔴"
"""

# Write to file using UTF-8
with open("agents/decision_agent.py", "w", encoding="utf-8") as f:
    f.write(code)

print("decision_agent.py created successfully!")


decision_agent.py created successfully!


In [4]:
# agents/data_preprocessing_agent.py
import os

# Make sure folder exists
os.makedirs("agents", exist_ok=True)

# Agent code
code = """
import numpy as np

class DataPreprocessingAgent:
    def __init__(self):
        pass

    def clean_input(self, data_dict):
        cleaned = {}
        for key, val in data_dict.items():
            if val is None or val < 0:
                cleaned[key] = 0
            else:
                cleaned[key] = val
        return cleaned

if __name__ == "__main__":
    agent = DataPreprocessingAgent()
    sample_data = {'temperature': 25, 'humidity': -5, 'no2': None}
    cleaned = agent.clean_input(sample_data)
    print("Cleaned data:", cleaned)
"""

# Save to file
with open("agents/data_preprocessing_agent.py", "w") as f:
    f.write(code)

print("Agent saved successfully!")

Agent saved successfully!


In [5]:
# Verify 
import os

print("Files in agents/:")
print(os.listdir("agents"))


Files in agents/:
['.ipynb_checkpoints', 'data_preprocessing_agent.py', 'decision_agent.py', 'prediction_agent.py', '__pycache__']


In [11]:
# Use NumPy arrays consistently (simplest):
import joblib
import pandas as pd

# Load the saved PM10 model
model = joblib.load(r"C:\Users\Lenovo\Desktop\Air_Quality_Prediction\notebooks\final_pm10_model.pkl")


# Example input (must match the feature count/order your model expects)
import numpy as np

# Suppose your model expects 13 features
input_array = np.array([[30, 50, 40, 20, 5, 0.8, 10, 25, 3, 180, 1012, 1, 4]])
predicted_pm2_5 = model.predict(input_array)

print("Predicted PM2_5:", predicted_pm10[0])

Predicted PM2_5: -833.1935092848082


In [12]:
# Install Required Library
!pip install ipywidgets

In [27]:
# -----------------------------
# PM2.5 Prediction Dashboard
# -----------------------------

import ipywidgets as widgets
from IPython.display import display, clear_output
import importlib, os, pandas as pd, datetime
import numpy as np
import joblib

# -----------------------------
# Load agents
# -----------------------------
from agents import data_preprocessing_agent, decision_agent
importlib.reload(data_preprocessing_agent)
importlib.reload(decision_agent)

from agents.data_preprocessing_agent import DataPreprocessingAgent
from agents.decision_agent import DecisionAgent

# -----------------------------
# PredictionAgent with correct scaling and realistic defaults
# -----------------------------
class PredictionAgent:
    def __init__(self, model_path, scaler_path=None):
        self.model = joblib.load(model_path)
        self.scaler = joblib.load(scaler_path) if scaler_path else None
        # Only scale first 5 features
        self.scaled_features_indices = [0,1,2,3,4]

    def predict_pm25(self, features):
        features = np.array(features).reshape(1, -1)

        if self.scaler:
            # Convert only scaled features to DataFrame (avoid warnings)
            df = pd.DataFrame(features[:, self.scaled_features_indices],
                              columns=['CO','NO2','SO2','PM2_5_lag1','NO2_lag1'])
            features[:, self.scaled_features_indices] = self.scaler.transform(df)

        prediction = self.model.predict(features)[0]
        return float(prediction)

# -----------------------------
# Input widgets
# -----------------------------
temperature = widgets.FloatText(value=22, description='🌡 Temperature (°C)', layout=widgets.Layout(width='250px'))
humidity = widgets.FloatText(value=45, description='💧 Humidity (%)', layout=widgets.Layout(width='250px'))
no2 = widgets.FloatText(value=10, description='🧪 NO₂ (µg/m³)', layout=widgets.Layout(width='250px'))
predict_button = widgets.Button(description='✨ Predict PM2.5', button_style='success', layout=widgets.Layout(width='200px'))
output = widgets.Output()

# -----------------------------
# Initialize agents
# -----------------------------
preprocessor = DataPreprocessingAgent()
model_path = os.path.join("..","notebooks","final_pm2_5_model.pkl")
scaler_path = os.path.join("..","notebooks","scaler_pm2_5.pkl")
predictor = PredictionAgent(model_path, scaler_path)
decision_agent = DecisionAgent()

# -----------------------------
# Button click logic
# -----------------------------
def on_predict_clicked(b):
    with output:
        clear_output()
        
        # --- Preprocess user input ---
        raw_input = {'temperature': temperature.value,
                     'humidity': humidity.value,
                     'no2': no2.value}
        cleaned_input = preprocessor.clean_input(raw_input)
        
        # --- Prepare features (realistic defaults) ---
        now = datetime.datetime.now()
        features = [
            0.7,                        # CO realistic default
            cleaned_input['no2'],       # NO2 from UI
            8.0,                        # SO2 realistic default
            5.0,                        # PM2_5_lag1 realistic default
            cleaned_input['no2'],       # NO2_lag1
            now.year,
            now.month,
            now.weekday()
        ]
        
        # --- Prediction ---
        pm2_5_value = predictor.predict_pm25(features)
        aqi_status = decision_agent.classify_aqi(pm2_5_value)
        recommendation = decision_agent.get_recommendation(aqi_status)
        
        # --- Display professional + cute card ---
        display(widgets.HTML(f"""
        <div style="
            background:#f8f9fa;
            padding:20px;
            border-radius:12px;
            border-left:6px solid #4CAF50;
            box-shadow:0px 4px 10px rgba(0,0,0,0.08);
            font-family:Arial;
        ">
            <h3>📊 Air Quality Analysis Result</h3>
            <hr>
            <p><b>PM2.5 Level:</b> <span style="color:#2E86C1;">{pm2_5_value:.2f} µg/m³</span></p>
            <p><b>AQI Status:</b> <span style="color:#28a745;">{aqi_status}</span></p>
            <p><b>Recommendation:</b></p>
            <p style="margin-left:15px;">💡 {recommendation}</p>
        </div>
        """))
        
        # --- Save prediction ---
        new_data = pd.DataFrame([{
            "temperature": cleaned_input['temperature'],
            "humidity": cleaned_input['humidity'],
            "no2": cleaned_input['no2'],
            "pm2_5_pred": pm2_5_value,
            "aqi_status": aqi_status,
            "recommendation": recommendation
        }])
        
        file_path = "predictions.csv"
        if os.path.exists(file_path):
            new_data.to_csv(file_path, mode='a', header=False, index=False)
        else:
            new_data.to_csv(file_path, index=False)

# -----------------------------
# Connect button & display UI
# -----------------------------
predict_button.on_click(on_predict_clicked)

display(
    widgets.VBox([
        widgets.HTML("<h2 style='color:#2c3e50;'>🌍 PM2.5 Air Quality Prediction Dashboard</h2>"),
        widgets.HTML("<p style='color:#555;'>Enter environmental conditions below to estimate PM2.5 concentration using an AI-based agent system.</p>"),
        widgets.HBox([temperature, humidity, no2]),
        widgets.HTML("<br>"),
        predict_button,
        widgets.HTML("<br><br>"),
        output
    ])
)
